## Gradio
This is a tool that allows us to create nice user interfaces for our LLMs. It simplifies the task of deploying them by removing the need to understand web development in depth.

In [1]:
import os
import gradio as gr
import google.generativeai as palm
from IPython.display import Markdown

# Custom modules
from embeddings_palm import get_palm_embeddings

In [2]:
api_key = os.getenv("PALM_API_KEY")
palm.configure(api_key=api_key)

## Pinecone

In [3]:
import pinecone

### Credentials

In [4]:
pinecone_api_key = os.getenv('PINECONE_API_KEY')

### Connecting to Pinecone Index

In [5]:

index_name = 'work-documents'

# initialize connection to pinecone
pinecone.init(
    api_key= pinecone_api_key,
    environment="gcp-starter"  # next to API key in console
)

In [6]:
# connect to index
index = pinecone.GRPCIndex(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.00046,
 'namespaces': {'': {'vector_count': 46}},
 'total_vector_count': 46}

In [13]:
def make_query(query):
    # Get query vector embeddings
    xq = get_palm_embeddings(query)
    res = index.query(xq, top_k=5, include_metadata=True)

    # get list of retrieved text
    contexts = [item['metadata']['text'] for item in res['matches']]

    # Concatenate retrieved texts from vector database with the query
    ## May exceed context length if too many
    augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query 
    
    # system message to 'prime' the model
    primer = f"""
    You are a tasker working on a labelling project on an online site called Remotasks \
    These task types are categorized under 3D and are lidar labelling tasks \
    You will be provided with parts of the guidelines describing how to label various objects in the task \
    You MUST be detail-oriented because slight the smallest misreadings of the instructions could lead to very wrong labelling \
    If for some reason you cannot find text relevant to this kind of task labelling say 'I DONT KNOW' \
    Do not make guesses or speculate, only say things that have factual basis in the guidelines \
    Please quote snippets from the guidelines to support your points in each of your responses \
    The response should look as follows, whenever possible: \

    `
    Your summary of the response
    "<Quote from the guidelines in double quotation marks>"
    `

    A query will be provided after the task guidelines for you to answer \
    """
    res = palm.chat(context=primer, messages=augmented_query, temperature=0.0)
    return res.last


In [ ]:
gr.Marown()kd

In [16]:
demo = gr.Interface(fn=make_query, 
                    inputs=[gr.Textbox(label="Query to make", lines=6)],
                    outputs=[gr.Textbox(label="Result", lines=3)],
                    title="RAG for Work Documents",
                    description="Query documents stored in a vector database and have them summarized"
                   )
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://69de3f5720883a2031.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [9]:
gr.close_all()

Closing server running on port: 7860
